# Extracting columns from the FIES

The following strings  columns are streight forward

HHID = ''
weight = ''
income = ''

Critical are income_sp and vul and region

## Columns for social spending

4Ps columns:

How much did you receive in total from the 4Ps from January to June 2015? 

jandec-amount

Pension (Government)

pensiongqty

Pension (Private)

pensionpqty

Survivor’s Benefit

survivorqty

National Health Insurance Program (Philhealth)

philhealthqty

Training for Work Scholarship Program (TWSP)

twspqty

Agrarian Reform Community Development Program (ARCDP)

arcdpqty

Community-Based Employment Program (CBEP)

cbepqty

Disability Benefit

disabilityqty 

Scholarship Benefits and Student Financial Assistance (Government)

govtqty

Scholarship Benefits and Student Financial Assistance (Private)

privqty

Others: 

othersqty


--> we assume that all the quantity given induvidually is more or less summarized in columns (Problem: nothing is given in the individual cells and something is given in the following columns):

    t930510 - Pension and retirement total
    
    t930520 - Workmen's Compensation total
    
    t930530 - Social security benefits (total)
    
    t930220 - Total government institutions
    
Additionally we consider

    t930210 - Total other families
    
    t930230 - Total private institutions


In [130]:
import pandas as pd
import numpy as np
import random
sta = pd.read_stata('/home/insauer/projects/WB_model/hhwb/data/FIES2015_v2.dta',convert_categoricals=False)


In [174]:
hhids = np.arange(sta.shape[0])
sta['hhid']=hhids.astype(int)

In [132]:
test =list(set(sta['w_regn']))
test
#str_to_int2 = [int(x) for x in test]

['12',
 '41',
 '02',
 '09',
 '06',
 '01',
 '14',
 '16',
 '05',
 '03',
 '10',
 '08',
 '13',
 '42',
 '11',
 '15',
 '07']

In [3]:
#str_to_int2.sort()
#str_to_int2

In [175]:
sta_income = sta[['hhid','w_regn','urb', 'rfact', 'pov_ind', 'pcinc_decile','toinc_decile','urban_poverty',
                  'rural_poverty','pcinc','toinc','ttotdis','walls',
                 't930510', 't930520', 't930530', 't930220', 't930210',
                 't930230']]
sta_income = sta_income.replace([np.nan],[0])

sta_income['income_sp'] = sta_income[['t930510', 't930520', 't930530', 't930220', 't930210',
                 't930230']].sum(axis=1)
sta_income

,hhid,w_regn,urb,rfact,pov_ind,pcinc_decile,toinc_decile,urban_poverty,rural_poverty,pcinc,toinc,ttotdis,walls,t930510,t930520,t930530,t930220,t930210,t930230,income_sp
0,0,14,2,211.1572,2,9,9,0.0,21240.29,120083.00,480332,332646.0,1,12600,0,10800,0,15360,0,38760
1,1,14,2,211.1572,2,7,6,0.0,21240.29,66078.33,198235,185266.0,1,0,0,0,0,0,0,0
2,2,14,2,211.1572,1,1,2,0.0,21240.29,13797.50,82785,92814.0,2,0,0,0,6600,3000,0,9600
3,3,14,2,211.1572,2,3,3,0.0,21240.29,23908.67,107589,111056.0,2,0,0,0,8800,0,0,8800
4,4,14,2,211.1572,2,6,6,0.0,21240.29,47330.50,189322,152047.0,4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41539,41539,12,2,392.6993,1,2,3,0.0,21030.95,18426.62,119773,84568.0,1,0,0,0,2400,5000,0,7400
41540,41540,12,2,392.6993,2,2,4,0.0,21030.95,22886.67,137320,70367.0,1,0,0,0,7000,4000,0,11000
41541,41541,12,2,392.6993,2,4,4,0.0,21030.95,33292.75,133171,66491.0,1,0,0,0,0,0,0,0
41542,41542,12,2,392.6993,1,1,4,0.0,21030.95,14388.89,129500,116547.0,1,0,0,0,3300,0,0,3300


In [176]:
def weight(x):
    w_sum = (x['rfact']*x['toinc']).sum()/1e6
    w_pop = x['rfact'].sum()
    return pd.Series([w_sum,
                     w_pop],
                     index=['w_sum_inc',
                            'w_pop'])

## Assign region ids 

01 --> I - Ilocos Region

02 --> II - Cagayan Valley

03 --> III - Central Luzon

05 --> V - Bicol Region

06 --> VI - Western Visayas

07 --> VII - Central Visayas

08 --> VIII - Eastern Visayas

09 --> IX - Zamboanga Peninsula

10 --> X - Northern Mindanao

11 --> XI - Davao Region

12 --> XII - SOCCSKSARGEN

13 --> National Capital Region

14 --> Cordillera Administrative Region

15 --> Autonomous Region in Muslim Mindanao

16 --> XIII - Caraga

41 --> IVA - CALABARZON

42 --> IVB - MIMAROPA


In [177]:
regsta_income = sta_income.groupby(['w_regn']).apply(weight).reset_index()
regsta_income['w_mean'] = regsta_income['w_sum_inc']/regsta_income['w_pop']
regsta_income[['w_regn','w_mean','w_sum_inc', 'w_pop']]

,w_regn,w_mean,w_sum_inc,w_pop
0,01,0.238064,2.785151e+05,1.169919e+06
1,02,0.237216,1.935888e+05,8.160874e+05
2,03,0.299154,7.500540e+05,2.507250e+06
3,05,0.187446,2.364762e+05,1.261569e+06
4,06,0.225977,3.840394e+05,1.699459e+06
5,07,0.239054,3.997343e+05,1.672148e+06
6,08,0.196524,1.917199e+05,9.755546e+05
7,09,0.190303,1.568724e+05,8.243313e+05
8,10,0.221493,2.279979e+05,1.029370e+06
9,11,0.246979,2.855597e+05,1.156213e+06


## Vunerability selection

We make use of the question:

What type of construction materials are the outer walls made of?

1 - Strong materials
(galvanized iron, aluminum, tile, concrete, brick, stone, wood, plywood, asbestos)

2 - Light materials (bamboo, sawali, cogon, nipa, anahaw)

3 - Salvaged/makeshift materials

4 - Mixed but predominantly strong materials

5 - Mixed but predominantly light materials

6 - Mixed but predominantly salvaged materials

7 - Not applicable

We assign vulnerabilities similar to Walsh et al 2019






| FIES Discriptor | Category | Indicator | Vulnerability |
|---|---|---|---|
|Strong material (galvanized iron, brick, tile, concrete, aluminum, stone, asbestos)| robust|1|0.14 ± 0.06|
|Mixed but predominantly strong materials|robust|4|0.14 ± 0.06|
|Light material (cogon, nipa, anahaw)|moderate|2|0.40 ± 0.08|
|Mixed but predominantly salvaged materials|moderate|5|0.40 ± 0.08|
|Salvaged/makeshift materials|fragile|3|0.70 ± 0.14|
|Mixed but predominantly salvaged materials|fragile|6|0.70 ± 0.14|
|Not Applicable|fragile|7|0.70 ± 0.14|




In [178]:
vul_cor =[100, 0.14, 0.4, 0.7, 0.14, 0.4, 0.7, 0.7]
vul_alt =[100, 0.06, 0.08, 0.14, 0.06, 0.08, 0.14, 0.14]
sta_income['walls'].tolist()
vul_list = [random.uniform(vul_cor[i]-vul_alt[i], vul_cor[i]+vul_alt[i]) for i in sta_income['walls'].tolist()]
sta_income['vul'] = vul_list
sta_income['vul']

0        0.176911
1        0.089133
2        0.433572
3        0.479037
4        0.129026
           ...   
41539    0.155244
41540    0.142635
41541    0.196871
41542    0.159823
41543    0.182282
Name: vul, Length: 41544, dtype: float64

In [179]:
hh = random.uniform(0.08, 0.2)
hh

0.11366346601467803

## Savings

We calculate the average gap (income less consumption) by region and
decile. We then assume that each household maintains one year’s surplus as precau-
tionary savings: separate from their productive assets, and available to be spent on
recovery or consumption smoothing.

In [180]:
## calculate annual surplus (income-disbursements)
sta_income['ann_surplus'] = sta_income['toinc'] - sta_income['ttotdis']

sta_income[['hhid','toinc_decile', 'toinc', 'ttotdis', 'ann_surplus']]

,hhid,toinc_decile,toinc,ttotdis,ann_surplus
0,0,9,480332,332646.0,147686.0
1,1,6,198235,185266.0,12969.0
2,2,2,82785,92814.0,-10029.0
3,3,3,107589,111056.0,-3467.0
4,4,6,189322,152047.0,37275.0
...,...,...,...,...,...
41539,41539,3,119773,84568.0,35205.0
41540,41540,4,137320,70367.0,66953.0
41541,41541,4,133171,66491.0,66680.0
41542,41542,4,129500,116547.0,12953.0


In [181]:
sta_decile = sta_income.groupby(['toinc_decile']).mean()
sta_decile[['toinc','income_sp','ttotdis','ann_surplus', 'vul']]

,toinc,income_sp,ttotdis,ann_surplus,vul
toinc_decile,,,,,
1,55622.867799,10375.409716,57443.643594,-1820.775795,0.282113
2,87941.651353,13251.400671,87001.107195,940.544158,0.257414
3,111730.247208,14137.737973,106818.988187,4911.259021,0.246796
4,135580.190056,15322.597740,127803.479322,7776.710734,0.228868
5,163325.082150,17106.112689,150673.504735,12651.577415,0.205221
6,198941.002008,18700.338269,181451.297867,17489.704141,0.188732
7,248499.046340,20314.600579,223025.755661,25473.290679,0.170216
8,318910.086667,24601.122133,283900.749600,35009.337067,0.159554
9,439584.002446,30134.051101,391030.248165,48553.754281,0.151562


In [182]:
## we remove the negative savings

sta_decile.loc[sta_decile['ann_surplus']<0,'ann_surplus'] = 0.
sta_decile['ann_surplus']

toinc_decile
1          0.000000
2        940.544158
3       4911.259021
4       7776.710734
5      12651.577415
6      17489.704141
7      25473.290679
8      35009.337067
9      48553.754281
10    132688.289357
Name: ann_surplus, dtype: float64

In [183]:
sta_decile['ann_surplus'].iloc[0]

0.0

In [184]:
savings = [sta_decile['ann_surplus'].iloc[i-1] for i in sta_income['toinc_decile'].tolist()]
sta_income['savings']= savings
sta_income[['hhid','toinc_decile','toinc','income_sp','ttotdis','ann_surplus','walls', 'vul','savings']]

,hhid,toinc_decile,toinc,income_sp,ttotdis,ann_surplus,walls,vul,savings
0,0,9,480332,38760,332646.0,147686.0,1,0.176911,48553.754281
1,1,6,198235,0,185266.0,12969.0,1,0.089133,17489.704141
2,2,2,82785,9600,92814.0,-10029.0,2,0.433572,940.544158
3,3,3,107589,8800,111056.0,-3467.0,2,0.479037,4911.259021
4,4,6,189322,0,152047.0,37275.0,4,0.129026,17489.704141
...,...,...,...,...,...,...,...,...,...
41539,41539,3,119773,7400,84568.0,35205.0,1,0.155244,4911.259021
41540,41540,4,137320,11000,70367.0,66953.0,1,0.142635,7776.710734
41541,41541,4,133171,0,66491.0,66680.0,1,0.196871,7776.710734
41542,41542,4,129500,3300,116547.0,12953.0,1,0.159823,7776.710734


## Get final dataset

In [185]:
final_data = sta_income[['hhid','w_regn', 'urb', 'rfact','pov_ind', 'toinc_decile',
                         'urban_poverty', 'rural_poverty', 'toinc','income_sp',
                         'vul','savings']]
final_data

,hhid,w_regn,urb,rfact,pov_ind,toinc_decile,urban_poverty,rural_poverty,toinc,income_sp,vul,savings
0,0,14,2,211.1572,2,9,0.0,21240.29,480332,38760,0.176911,48553.754281
1,1,14,2,211.1572,2,6,0.0,21240.29,198235,0,0.089133,17489.704141
2,2,14,2,211.1572,1,2,0.0,21240.29,82785,9600,0.433572,940.544158
3,3,14,2,211.1572,2,3,0.0,21240.29,107589,8800,0.479037,4911.259021
4,4,14,2,211.1572,2,6,0.0,21240.29,189322,0,0.129026,17489.704141
...,...,...,...,...,...,...,...,...,...,...,...,...
41539,41539,12,2,392.6993,1,3,0.0,21030.95,119773,7400,0.155244,4911.259021
41540,41540,12,2,392.6993,2,4,0.0,21030.95,137320,11000,0.142635,7776.710734
41541,41541,12,2,392.6993,2,4,0.0,21030.95,133171,0,0.196871,7776.710734
41542,41542,12,2,392.6993,1,4,0.0,21030.95,129500,3300,0.159823,7776.710734


In [186]:
final_data.loc[final_data['urb']==1, 'poverty_line'] = final_data.loc[final_data['urb']==1,'urban_poverty']
final_data.loc[final_data['urb']==2, 'poverty_line'] = final_data.loc[final_data['urb']==2,'rural_poverty']
final_data[2080:3000]

final_data = final_data.drop(columns=['urban_poverty', 'rural_poverty'])
final_data

/home/insauer/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/insauer/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,hhid,w_regn,urb,rfact,pov_ind,toinc_decile,toinc,income_sp,vul,savings,poverty_line
0,0,14,2,211.1572,2,9,480332,38760,0.176911,48553.754281,21240.29
1,1,14,2,211.1572,2,6,198235,0,0.089133,17489.704141,21240.29
2,2,14,2,211.1572,1,2,82785,9600,0.433572,940.544158,21240.29
3,3,14,2,211.1572,2,3,107589,8800,0.479037,4911.259021,21240.29
4,4,14,2,211.1572,2,6,189322,0,0.129026,17489.704141,21240.29
...,...,...,...,...,...,...,...,...,...,...,...
41539,41539,12,2,392.6993,1,3,119773,7400,0.155244,4911.259021,21030.95
41540,41540,12,2,392.6993,2,4,137320,11000,0.142635,7776.710734,21030.95
41541,41541,12,2,392.6993,2,4,133171,0,0.196871,7776.710734,21030.95
41542,41542,12,2,392.6993,1,4,129500,3300,0.159823,7776.710734,21030.95


In [187]:
final_data.loc[final_data['urb']==1, 'isurban'] = int(1)
final_data.loc[final_data['urb']==2, 'isurban'] = int(0)

final_data.loc[final_data['pov_ind']==1, 'ispoor'] = int(1)
final_data.loc[final_data['pov_ind']==2, 'ispoor'] = int(0)

final_data = final_data.drop(columns=['urb', 'pov_ind'])
final_data


,hhid,w_regn,rfact,toinc_decile,toinc,income_sp,vul,savings,poverty_line,isurban,ispoor
0,0,14,211.1572,9,480332,38760,0.176911,48553.754281,21240.29,0.0,0.0
1,1,14,211.1572,6,198235,0,0.089133,17489.704141,21240.29,0.0,0.0
2,2,14,211.1572,2,82785,9600,0.433572,940.544158,21240.29,0.0,1.0
3,3,14,211.1572,3,107589,8800,0.479037,4911.259021,21240.29,0.0,0.0
4,4,14,211.1572,6,189322,0,0.129026,17489.704141,21240.29,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
41539,41539,12,392.6993,3,119773,7400,0.155244,4911.259021,21030.95,0.0,1.0
41540,41540,12,392.6993,4,137320,11000,0.142635,7776.710734,21030.95,0.0,0.0
41541,41541,12,392.6993,4,133171,0,0.196871,7776.710734,21030.95,0.0,0.0
41542,41542,12,392.6993,4,129500,3300,0.159823,7776.710734,21030.95,0.0,1.0


In [188]:
final_data['isurban']= final_data['isurban'].astype(int)
final_data['ispoor']= final_data['ispoor'].astype(int)
final_data

,hhid,w_regn,rfact,toinc_decile,toinc,income_sp,vul,savings,poverty_line,isurban,ispoor
0,0,14,211.1572,9,480332,38760,0.176911,48553.754281,21240.29,0,0
1,1,14,211.1572,6,198235,0,0.089133,17489.704141,21240.29,0,0
2,2,14,211.1572,2,82785,9600,0.433572,940.544158,21240.29,0,1
3,3,14,211.1572,3,107589,8800,0.479037,4911.259021,21240.29,0,0
4,4,14,211.1572,6,189322,0,0.129026,17489.704141,21240.29,0,0
...,...,...,...,...,...,...,...,...,...,...,...
41539,41539,12,392.6993,3,119773,7400,0.155244,4911.259021,21030.95,0,1
41540,41540,12,392.6993,4,137320,11000,0.142635,7776.710734,21030.95,0,0
41541,41541,12,392.6993,4,133171,0,0.196871,7776.710734,21030.95,0,0
41542,41542,12,392.6993,4,129500,3300,0.159823,7776.710734,21030.95,0,1


In [189]:
final_data = final_data.rename(columns={'w_regn':'region',
                                        'rfact': 'weight',
                                        'toinc_decile': 'decile',
                                        'toinc':'income'})

final_data

,hhid,region,weight,decile,income,income_sp,vul,savings,poverty_line,isurban,ispoor
0,0,14,211.1572,9,480332,38760,0.176911,48553.754281,21240.29,0,0
1,1,14,211.1572,6,198235,0,0.089133,17489.704141,21240.29,0,0
2,2,14,211.1572,2,82785,9600,0.433572,940.544158,21240.29,0,1
3,3,14,211.1572,3,107589,8800,0.479037,4911.259021,21240.29,0,0
4,4,14,211.1572,6,189322,0,0.129026,17489.704141,21240.29,0,0
...,...,...,...,...,...,...,...,...,...,...,...
41539,41539,12,392.6993,3,119773,7400,0.155244,4911.259021,21030.95,0,1
41540,41540,12,392.6993,4,137320,11000,0.142635,7776.710734,21030.95,0,0
41541,41541,12,392.6993,4,133171,0,0.196871,7776.710734,21030.95,0,0
41542,41542,12,392.6993,4,129500,3300,0.159823,7776.710734,21030.95,0,1


In [193]:
## save it
final_data.to_csv('hhwb/data/FIES_prepared/survey_PHL.csv')